In [1]:
import pandas as pd
import numpy as np
import os
import nltools as nlt
import nilearn as nil
import nibabel as nib
import warnings
import glob
import random
import pickle
from operator import itemgetter

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


This file loads behavioral and neural data , does a test-train split, and saves it to a dump file. It's deprecated and replaced by `load_filter_test_train_split.ipynb` and `load_multisubject_brain_data.ipynb`.

In [2]:
#load behavioral data

import dev_wtp_io_utils
behavioral_data_path = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/Tasks/WTP/output/'
[run_list, run_event_df_list, run_event_stage_df_list] = dev_wtp_io_utils.get_comprehensive_run_data_from_mat_dir(behavioral_data_path)



subject 040, subject 039, subject 168, subject 063, subject 129, subject 067, subject 150, subject 159, 
SUBJ 159, 2, run14
no data for this run.
subject 004, subject 091, subject 180, subject 008, subject 095, subject 099, subject 032, 
SUBJ 032, 2, run2
no data for this run.
subject 036, subject 171, subject 100, 
SUBJ 100, 3, run2
no data for this run.
subject 109, subject 041, subject 060, subject 059, subject 130, subject 064, subject 068, subject 001, subject 087, subject 192, subject 005, subject 121, subject 999, 
SUBJ 999, 2, run1
no data for this run.


SUBJ 999, 2, run2
Data for this run is not in the expected format or is missing. Skipping this run.
time: 29-Oct-2020 17:09:09

StartTime: 3035.874545661

Jitter: array of length 16

TrialStart: array of length 2

ISI: array of length 2

FoodOn: array of length 2

BidOn: array of length 2

FoodOnset: array of length 2

BidOnset: array of length 2

FoodDuration: array of length 2

BidDuration: array of length 2

FoodPic: array 

In [3]:
#load design data from the official list of task design.
#this is NOT per subject.
betas = pd.read_csv("../data/DEV QC and Exclusions - betas_machine_readable.csv")
wtp_betas = betas[betas.task=='WTP betas']
#get the betas just for the four runs
wtp_betas_runfiles = wtp_betas[[bt in ['run1','run2','run3','run4'] for bt in wtp_betas.type]].copy()
#number them
wtp_betas_runfiles['event_id'] = wtp_betas_runfiles.groupby('type').cumcount()+1

In [4]:
wtpw1_behavdesign = wtp_betas_runfiles.merge(run_event_df_list, how='left',
                         left_on=['type','event_id'],
                        right_on=['run','event_id'])

wtpw1_behavdesign = wtpw1_behavdesign[(wtpw1_behavdesign.wave==1)]

wtpw1_behavdesign.sample(5)


,beta,type,task,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
28526,beta_0071.nii,run4,WTP betas,8,2.083661,84.925639,6.525462,JackLinks.bmp,8,unhealthy_disliked,unhealthy,disliked,1,5,1.584060,91.451101,run4,1,DEV107
26529,beta_0065.nii,run4,WTP betas,2,3.000000,18.684837,6.530201,RXBar.bmp,2,healthy_liked,healthy,liked,2,5,1.780283,25.215038,run4,1,DEV084
1813,beta_0003.nii,run1,WTP betas,3,1.780283,30.073052,6.532024,BabybelCheese.bmp,3,healthy_liked,healthy,liked,4,NULL,0.095117,36.605076,run1,1,DEV114
24392,beta_0054.nii,run3,WTP betas,12,0.925946,127.241403,6.531194,SourPunch.bmp,12,unhealthy_disliked,unhealthy,disliked,1,5,0.670865,133.772597,run3,1,DEV113
20643,beta_0043.nii,run3,WTP betas,1,3.000000,6.090674,6.548668,rugercookie.bmp,1,unhealthy_liked,unhealthy,liked,2,6,3.000000,12.639341,run3,1,DEV186


In [5]:
#iterate through subject directories.

subject_dirs_all = glob.glob('/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/betaseries/sub-DEV*')
subject_dirs_all.sort()

In [6]:
#make some labels based on this list
import re
sub_labels_all = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in subject_dirs_all]
sub_ids_all = [(int)(re.search("DEV(\\d\\d\\d)",sid)[1]) for sid in subject_dirs_all]

## Deal with incomplete data

1. Events listed in betas that do not have a betas file

2. Subjects listed in subject_dirs that don't have corresponding beta data

In [7]:
wtpw1_behavdesign

,beta,type,task,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
3,beta_0001.nii,run1,WTP betas,1,3.000000,6.080660,6.564707,SourPunch.bmp,1,unhealthy_liked,unhealthy,liked,4,NULL,3.0,12.645367,run1,1,DEV040
6,beta_0001.nii,run1,WTP betas,1,3.000000,6.068091,6.548297,M&Ms.bmp,1,unhealthy_liked,unhealthy,liked,4,6,3.0,12.616389,run1,1,DEV039
9,beta_0001.nii,run1,WTP betas,1,3.000000,6.076286,6.548571,MacNCheese.bmp,1,unhealthy_liked,unhealthy,liked,3,8,3.0,12.624857,run1,1,DEV168
13,beta_0001.nii,run1,WTP betas,1,3.000000,6.085906,6.553895,FamousAmos.bmp,1,unhealthy_liked,unhealthy,liked,2,6,3.0,12.639801,run1,1,DEV063
16,beta_0001.nii,run1,WTP betas,1,3.000000,6.068752,6.549177,PeanutButterCup.bmp,1,unhealthy_liked,unhealthy,liked,4,NULL,3.0,12.617930,run1,1,DEV129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31399,beta_0079.nii,run4,WTP betas,16,1.421652,168.783176,6.532094,lunchables_turkey.bmp,16,unhealthy_liked,unhealthy,liked,1,NULL,NaN,175.315270,run4,1,DEV179
31400,beta_0079.nii,run4,WTP betas,16,1.421652,168.726133,6.532007,FamousAmos.bmp,16,unhealthy_liked,unhealthy,liked,3,6,NaN,175.258140,run4,1,DEV098
31403,beta_0079.nii,run4,WTP betas,16,1.421652,168.782857,6.533177,SourCreamChips.bmp,16,unhealthy_liked,unhealthy,liked,4,7,NaN,175.316034,run4,1,DEV108
31404,beta_0079.nii,run4,WTP betas,16,1.421652,168.644515,6.516474,Poptart1.bmp,16,unhealthy_liked,unhealthy,liked,2,5,NaN,175.160989,run4,1,DEV035


In [8]:
pd.set_option('display.max_rows', 50)

Exclude from consideration subjects that have too few events (i.e., less than 64).

In [9]:
events_per_subject = wtpw1_behavdesign.groupby('subject').duration.count()
subjects_with_too_few_events = events_per_subject[events_per_subject<64].index.tolist()

wtpw1_behavdesign_clean = wtpw1_behavdesign.loc[[(s in subjects_with_too_few_events)==False for s in wtpw1_behavdesign.subject]]


In [10]:
print(wtpw1_behavdesign_clean.shape, wtpw1_behavdesign.shape)

(11648, 19) (11824, 19)


The following subjects have behavioral data recorded in the MATLAB behavioral output folder via the function `get_comprehensive_run_data_from_mat_dir`, but they do not have a beta.nii folder:

In [11]:

subjects_with_behavioral_no_beta_folder = set(wtpw1_behavdesign_clean.subject).difference(set(sub_labels_all))
subjects_with_behavioral_no_beta_folder = list(subjects_with_behavioral_no_beta_folder)
subjects_with_behavioral_no_beta_folder.sort()
print( subjects_with_behavioral_no_beta_folder)

['DEV002', 'DEV007', 'DEV011', 'DEV020', 'DEV032', 'DEV047', 'DEV063', 'DEV064', 'DEV067', 'DEV075', 'DEV078', 'DEV088', 'DEV090', 'DEV093', 'DEV094', 'DEV095', 'DEV096', 'DEV097', 'DEV098', 'DEV099', 'DEV100', 'DEV101', 'DEV102', 'DEV103', 'DEV104', 'DEV105', 'DEV106', 'DEV107', 'DEV108', 'DEV109', 'DEV110', 'DEV111', 'DEV112', 'DEV113', 'DEV115', 'DEV116', 'DEV117', 'DEV118', 'DEV119', 'DEV120', 'DEV121', 'DEV122', 'DEV123', 'DEV124', 'DEV126', 'DEV127', 'DEV128', 'DEV129', 'DEV130', 'DEV132', 'DEV133', 'DEV134', 'DEV135', 'DEV137', 'DEV138', 'DEV139', 'DEV140', 'DEV141', 'DEV143', 'DEV144', 'DEV145', 'DEV147', 'DEV149', 'DEV150', 'DEV151', 'DEV152', 'DEV153', 'DEV154', 'DEV155', 'DEV156', 'DEV157', 'DEV158', 'DEV159', 'DEV161', 'DEV163', 'DEV164', 'DEV166', 'DEV167', 'DEV168', 'DEV169', 'DEV170', 'DEV171', 'DEV173', 'DEV174', 'DEV176', 'DEV177', 'DEV178', 'DEV179', 'DEV180', 'DEV181', 'DEV182', 'DEV183', 'DEV185', 'DEV186', 'DEV187', 'DEV189', 'DEV190', 'DEV191', 'DEV192', 'DEV193',

The following subjects have a `beta.nii` folder but they do not have behavioral data recorded in the MATLAB behavioral output folder via the function `get_comprehensive_run_data_from_mat_dir`.

See: https://docs.google.com/presentation/d/1K-nFrZYE6rR8t0myNyacB7frBzV3B1--nMqPhVkwL8E/edit#slide=id.gd9fcc4129a_0_0

In [63]:
behavioral_data_subjects = set(wtpw1_behavdesign_clean.subject)

In [64]:
subjects_with_beta_folder_no_behavioral = set(sub_labels_all).difference(behavioral_data_subjects)

In [65]:
subjects_with_beta_folder_no_behavioral

{'DEV072', 'DEV082'}

Finally we want to remove those subjects with no behavioral data from the list...

In [55]:
subject_dirs_usable = [d for d, sl in zip(subject_dirs_all,sub_labels_all) if sl not in subjects_with_beta_folder_no_behavioral]
sub_labels_usable = [sl for d, sl in zip(subject_dirs_all,sub_labels_all) if sl not in subjects_with_beta_folder_no_behavioral]

### Manage split and select subjects

In [16]:
# #make some labels based on this list
# import re
# sub_labels = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in subject_dirs]
# sub_ids = [(int)(re.search("DEV(\\d\\d\\d)",sid)[1]) for sid in subject_dirs]

In [17]:
brain_data_list = []

In [18]:
total_sample_size = 30
holdout_size = 5
holdout_n = (int)(total_sample_size/holdout_size)
holdout_vector = [x*holdout_n for x in range(0,holdout_n)]

In [19]:
total_sample_size = len(subject_dirs_usable)# - (len(subject_dirs_usable) % 5)
holdout_size = 5
holdout_n = (int)(total_sample_size/holdout_size)
holdout_vector = [x*holdout_size for x in range(0,holdout_n)]

In [20]:
holdout_n

15

In [21]:
print(total_sample_size,holdout_n,total_sample_size-holdout_n)

75 15 60


The only way to get a REALLY stable set of useable subjects, robust to even corrections to the dataset, is to make this list manually, so that's what we'll do.

In [44]:
import datetime
ts = datetime.datetime.now().strftime('%Y%m%dT%H%M%S')

In [46]:
ts

'20210601T172446'

In [57]:
random.seed(a=1622149277.6487303)
subj_labels_randomized = random.sample(sub_labels_usable,k=total_sample_size) # add an extra 5 as a hold-out
holdout_subjs = [x for i,x in enumerate(subj_labels_randomized) if i in holdout_vector]
train_subjs = [x for x in subj_labels_randomized if (x in holdout_subjs)==False]

In [58]:
pd.DataFrame(holdout_subjs).to_csv("../data/train_holdout_set_" + ts + "_holdout_subjs_15.csv",index=False)
pd.DataFrame(train_subjs).to_csv("../data/train_holdout_set_" + ts + "_train_subjs_75.csv",index=False)

In [59]:
#now load those back using a specific ts
holdout_subjs = list(pd.read_csv("../data/train_holdout_set_20210601T172446_holdout_subjs_15.csv").iloc[:,0])
train_subjs = list(pd.read_csv("../data/train_holdout_set_20210601T172446_train_subjs_75.csv").iloc[:,0])

In [60]:
len(train_subjs)

60

In [61]:
len(holdout_subjs)

15

In [62]:
def get_Brain_Data_for_sub(sub_label,behavdesign):
    subj_behav_design = behavdesign[[s in sub_label for s in behavdesign.subject]]
    del(behavdesign)
    
    if len(subj_behav_design)!=64:
        raise Exception("For "+sub_label+", expected 64 beta events but found " + str(len(subj_behav_design)))

    
    subject_dir = (
        '/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/betaseries/' +
        'sub-' + sub_label + '/'
    )
        
    #set path for the betas

    beta_path_sub049 = os.listdir(subject_dir)

    #iterate through the subjects
    # TO DO
    #iterate through and check all the betas exist
    for betafile in subj_behav_design.beta:
        betafilepath = subject_dir + betafile

        #check if file exists
        if os.path.exists(betafilepath):
            print('.',end='')
        else:
            raise Exception("beta " + betafile + ' does not exist')

    print("...verified that all expected betas exist! Processing...",flush=True,end='')



    #import them, but catch a very specific warning and don't show it because it gets annoying
    #https://docs.python.org/3/library/warnings.html
    with warnings.catch_warnings(record=True) as w:
        subj_data = nlt.Brain_Data([subject_dir + b for b in subj_behav_design.beta],
                                  X=subj_behav_design)
    na_inf_warn_count=0
    for wi in w:
        if wi.message.args[0]!='NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.':
            warnings.warn(wi.message,type(wi))
        else:
            na_inf_warn_count+=1

    if na_inf_warn_count>0:
        print("caught "+ str(na_inf_warn_count) + " instances of the warning:\n" + 
             'NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.')
        
    print('...imported data.')
    return(subj_data)



## Run subject group

In [71]:
train_subjs.sort()

bd_dict={}
for sl in train_subjs:
    print(sl)
    bd = get_Brain_Data_for_sub(sl,wtpw1_behavdesign_clean)
    bd_dict[sl]=bd

DEV001
...................................................................verified that all expected betas exist! Processing...caught 64 instances of the warning:
NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
...imported data.
DEV004
...................................................................verified that all expected betas exist! Processing...caught 64 instances of the warning:
NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
...imported data.
DEV005
...................................................................verified that all expected betas exist! Processing...caught 64 instances of the warning:
NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
...imported data.
DEV006
...........................

...................................................................verified that all expected betas exist! Processing...caught 64 instances of the warning:
NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
...imported data.
DEV042
...................................................................verified that all expected betas exist! Processing...caught 64 instances of the warning:
NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
...imported data.
DEV043
...................................................................verified that all expected betas exist! Processing...caught 64 instances of the warning:
NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
...imported data.
DEV044
..................................

...................................................................verified that all expected betas exist! Processing...caught 64 instances of the warning:
NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
...imported data.
DEV083
...................................................................verified that all expected betas exist! Processing...caught 64 instances of the warning:
NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
...imported data.
DEV084
...................................................................verified that all expected betas exist! Processing...caught 64 instances of the warning:
NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
...imported data.
DEV085
..................................

In [72]:

Brain_Data_allsubs = list(bd_dict.values())[0]
for i in range(1,len(bd_dict.values())):
    print(i)
    val_to_append = list(bd_dict.values())[i]
    Brain_Data_allsubs= Brain_Data_allsubs.append(val_to_append)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [73]:
with open('../data/Brain_Data_75subs.pkl', 'wb') as pkl_file:
    pickle.dump(Brain_Data_allsubs,pkl_file)

### Specific subject check

In [58]:
pd.set_option('display.max_rows', 99)

In [59]:
subject_image_counts = wtpw1_behavdesign.subject.value_counts()
subject_image_counts = subject_image_counts.sort_index()
print(subject_image_counts)

DEV001    64
DEV002    64
DEV004    64
DEV005    64
DEV006    64
          ..
DEV197    64
DEV198    64
DEV206    64
DEV998    16
DEV999    64
Name: subject, Length: 187, dtype: int64


In [60]:
bd_dict={}
#for sl in sub_labels:
sl = 'DEV072'
print(sl)
bd = get_Brain_Data_for_sub(sl,wtpw1_behavdesign)
bd_dict[sl]=bd

DEV072


Exception: For DEV072, expected 64 beta events but found 0

In [ ]:
[subject_dir + b for b in subj_behav_design.beta]

In [ ]:

#import them, but catch a very specific warning and don't show it because it gets annoying
#https://docs.python.org/3/library/warnings.html
with warnings.catch_warnings(record=True) as w:
    subj_data = nlt.Brain_Data([subject_dir + b for b in subj_behav_design.beta],
                              X=subj_behav_design)

In [ ]:

na_inf_warn_count=0
for wi in w:
    if wi.message.args[0]!='NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.':
        warnings.warn(wi.message,type(wi))
    else:
        na_inf_warn_count+=1

if na_inf_warn_count>0:
    print("caught "+ str(na_inf_warn_count) + " instances of the warning:\n" + 
         'NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.')

print('...imported data.')
return(subj_data)

